In [2]:
from platform import python_version
print(python_version())
import pkg_resources
import sklearn
print(sklearn. __version__)
import pandas as pd
print(pd.__version__)
import pickle
import numpy as np

3.7.4
0.21.3
0.25.1


In [28]:
cgm_df = pd.read_csv("./CGMData.csv")
insulin_df = pd.read_csv("./InsulinData.csv")

In [10]:
cgm_df[['Date','Time','Sensor Glucose (mg/dL)']]

,Date,Time,Sensor Glucose (mg/dL)
0,2/12/2018,13:22:27,118.0
1,2/12/2018,13:17:27,122.0
2,2/12/2018,13:12:27,NaN
3,2/12/2018,13:07:27,NaN
4,2/12/2018,13:02:27,NaN
...,...,...,...
55338,7/25/2017,12:28:54,311.0
55339,7/25/2017,12:23:54,311.0
55340,7/25/2017,12:18:54,309.0
55341,7/25/2017,12:13:54,310.0


In [15]:
insulin_df[['Date','Time','Alarm']]

,Date,Time,Alarm
0,2/12/2018,13:20:53,NaN
1,2/12/2018,13:20:48,Other Alarm Code: 112
2,2/12/2018,13:18:48,NaN
3,2/12/2018,13:18:48,NaN
4,2/12/2018,13:12:33,NaN
...,...,...,...
41430,7/24/2017,19:00:01,NaN
41431,7/24/2017,18:59:44,NaN
41432,7/24/2017,18:59:44,NaN
41433,7/24/2017,18:59:44,NaN


In [ ]:
"""
a) Percentage time in hyperglycemia (CGM > 180 mg/dL),
b) percentage of time in hyperglycemia critical (CGM > 250 mg/dL),
c) percentage time in range (CGM >= 70 mg/dL and CGM <= 180 mg/dL),
d) percentage time in range secondary (CGM >= 70 mg/dL and CGM <= 150 mg/dL),
e) percentage time in hypoglycemia level 1 (CGM < 70 mg/dL), and
f) percentage time in hypoglycemia level 2 (CGM < 54 mg/dL)."""

In [58]:
switch = insulin_df[insulin_df['Alarm'] == "AUTO MODE ACTIVE PLGM OFF"][['Date','Time']].iloc[-1]

In [158]:
switch['Time']

'8:07:13'

In [70]:
index = cgm_df[(cgm_df['Date'] == switch['Date']) & (pd.to_datetime(cgm_df['Time'],format='%H:%M:%S').dt.time <= pd.to_datetime(switch['Time'],format='%H:%M:%S').time())][['Date','Time','Alarm']].index[0]
index

51087

In [71]:
auto_data = cgm_df.iloc[0:index]
manual_data = cgm_df.iloc[index:len(cgm_df)]

In [73]:
len(auto_data)

51087

In [74]:
len(manual_data)

4256

In [114]:
m_days = manual_data["Date"].unique()[::-1]

In [177]:
manual_hyperglycemia_critical_day, manual_hyperglycemia_critical_morning, manual_hyperglycemia_critical_night = 0, 0, 0
manual_hyperglycemia_day, manual_hyperglycemia_morning, manual_hyperglycemia_night = 0,0,0
manual_range_day, manual_range_morning, manual_range_night = 0, 0, 0
manual_range_secondary_day, manual_range_secondary_morning, manual_range_secondary_night = 0, 0, 0
manual_hypoglycemia_1_day, manual_hypoglycemia_1_morning, manual_hypoglycemia_1_night = 0, 0, 0
manual_hypoglycemia_2_day, manual_hypoglycemia_2_morning, manual_hypoglycemia_2_night = 0, 0, 0
total_manual = 0
total_manual_morning = 0
total_manual_night = 0

for day in m_days:
    #print(f"Day:{day}, Rows:{len(manual_data[(manual_data['Date'] == day) & (manual_data['Sensor Glucose (mg/dL)'].notna())])}")
    actual_night = len(manual_data[(manual_data['Date'] == day) & (pd.to_datetime(manual_data['Time'],format='%H:%M:%S').dt.time < pd.to_datetime("6:00:00",format='%H:%M:%S').time()) & (manual_data['Sensor Glucose (mg/dL)'].notna())])
    total_night = len(manual_data[(manual_data['Date'] == day) & (pd.to_datetime(manual_data['Time'],format='%H:%M:%S').dt.time < pd.to_datetime("6:00:00",format='%H:%M:%S').time())])
    night_diff = total_night - actual_night
    
    actual_morning = len(manual_data[(manual_data['Date'] == day) & (pd.to_datetime(manual_data['Time'],format='%H:%M:%S').dt.time >= pd.to_datetime("6:00:00",format='%H:%M:%S').time()) & (manual_data['Sensor Glucose (mg/dL)'].notna())])
    total_morning = len(manual_data[(manual_data['Date'] == day) & (pd.to_datetime(manual_data['Time'],format='%H:%M:%S').dt.time >= pd.to_datetime("6:00:00",format='%H:%M:%S').time())])
    morning_diff = total_morning - actual_morning
    
    actual_day = len(manual_data[(manual_data['Date'] == day) & (manual_data['Sensor Glucose (mg/dL)'].notna())])
    total_day = len(manual_data[(manual_data['Date'] == day)])
    day_diff = total_day - actual_day
        
    if day_diff <= 10 or total_day == 288:
        count = 1
        
        hyperglycemia_critical_day, hyperglycemia_critical_morning, hyperglycemia_critical_night = 0, 0, 0
        hyperglycemia_day, hyperglycemia_morning, hyperglycemia_night = 0,0,0
        range_day, range_morning, range_night = 0, 0, 0
        range_secondary_day, range_secondary_morning, range_secondary_night = 0, 0, 0
        hypoglycemia_1_day, hypoglycemia_1_morning, hypoglycemia_1_night = 0, 0, 0
        hypoglycemia_2_day, hypoglycemia_2_morning, hypoglycemia_2_night = 0, 0, 0
        
        data = manual_data[(manual_data['Date'] == day)].interpolate(limit=10)
        for level in data['Sensor Glucose (mg/dL)']:
            if level > 250:
                hyperglycemia_critical_day += 1
                if count <= 216: hyperglycemia_critical_morning += 1  
                else: hyperglycemia_critical_night += 1
            if level <= 250 and level > 180:
                hyperglycemia_day += 1
                if count <= 216: hyperglycemia_morning += 1 
                else: hyperglycemia_night += 1
            if level <= 180 and level >= 70:
                range_day += 1
                if count <= 216: range_morning += 1
                else: range_night += 1
            if level <= 150 and level >= 70:
                range_secondary_day += 1
                if count <= 216: range_secondary_morning += 1
                else: range_secondary_night += 1
            if level < 70 and level >= 54:
                hypoglycemia_1_day += 1
                if count <= 216: hypoglycemia_1_morning += 1
                else: hypoglycemia_1_night += 1
            if level < 54:
                hypoglycemia_2_day += 1
                if count <= 216: hypoglycemia_2_morning += 1
                else: hypoglycemia_2_night += 1
            count += 1
        manual_hyperglycemia_critical_day += hyperglycemia_critical_day/288
        manual_hyperglycemia_critical_morning += hyperglycemia_critical_morning/216
        manual_hyperglycemia_critical_night += hyperglycemia_critical_night/72
        manual_hyperglycemia_day += hyperglycemia_day/288
        manual_hyperglycemia_morning += hyperglycemia_morning/216
        manual_hyperglycemia_night += hyperglycemia_night/72
        manual_range_day += range_day/288
        manual_range_morning += range_morning/216
        manual_range_night += range_night/72
        manual_range_secondary_day += range_secondary_day/288
        manual_range_secondary_morning += range_secondary_morning/216
        manual_range_secondary_night += range_secondary_night/72
        manual_hypoglycemia_1_day += hypoglycemia_1_day/288
        manual_hypoglycemia_1_morning += hypoglycemia_1_morning/216
        manual_hypoglycemia_1_night += hypoglycemia_1_night/72
        manual_hypoglycemia_2_day += hypoglycemia_2_day/288
        manual_hypoglycemia_2_morning += hypoglycemia_2_morning/216
        manual_hypoglycemia_2_night += hypoglycemia_2_night/72
        total_manual += 1
        total_manual_morning += 1
        total_manual_night += 1
    elif morning_diff <= 10 or total_morning == 216:
        count = 1
        
        hyperglycemia_critical_day, hyperglycemia_critical_morning, hyperglycemia_critical_night = 0, 0, 0
        hyperglycemia_day, hyperglycemia_morning, hyperglycemia_night = 0,0,0
        range_day, range_morning, range_night = 0, 0, 0
        range_secondary_day, range_secondary_morning, range_secondary_night = 0, 0, 0
        hypoglycemia_1_day, hypoglycemia_1_morning, hypoglycemia_1_night = 0, 0, 0
        hypoglycemia_2_day, hypoglycemia_2_morning, hypoglycemia_2_night = 0, 0, 0
        
        data = manual_data[(manual_data['Date'] == day)& (pd.to_datetime(manual_data['Time'],format='%H:%M:%S').dt.time >= pd.to_datetime("6:00:00",format='%H:%M:%S').time())].interpolate(limit=10)
        for level in data['Sensor Glucose (mg/dL)']:
            if level > 250:
                hyperglycemia_critical_morning += 1  
            if level <= 250 and level > 180:
                hyperglycemia_morning += 1 
            if level <= 180 and level >= 70:
                range_morning += 1
            if level <= 150 and level >= 70:
                range_secondary_morning += 1
            if level < 70 and level >= 54:
                hypoglycemia_1_morning += 1
            if level < 54:
                hypoglycemia_2_morning += 1
            count += 1
        manual_hyperglycemia_critical_morning += hyperglycemia_critical_morning/216
        manual_hyperglycemia_morning += hyperglycemia_morning/216
        manual_range_morning += range_morning/216
        manual_range_secondary_morning += range_secondary_morning/216
        manual_hypoglycemia_1_morning += hypoglycemia_1_morning/216
        manual_hypoglycemia_2_morning += hypoglycemia_2_morning/216
        total_manual_morning += 1
    elif night_diff <= 10 or total_night == 72:
        count = 1
        
        hyperglycemia_critical_day, hyperglycemia_critical_morning, hyperglycemia_critical_night = 0, 0, 0
        hyperglycemia_day, hyperglycemia_morning, hyperglycemia_night = 0,0,0
        range_day, range_morning, range_night = 0, 0, 0
        range_secondary_day, range_secondary_morning, range_secondary_night = 0, 0, 0
        hypoglycemia_1_day, hypoglycemia_1_morning, hypoglycemia_1_night = 0, 0, 0
        hypoglycemia_2_day, hypoglycemia_2_morning, hypoglycemia_2_night = 0, 0, 0
        
        data = manual_data[(manual_data['Date'] == day) & (pd.to_datetime(manual_data['Time'],format='%H:%M:%S').dt.time < pd.to_datetime("6:00:00",format='%H:%M:%S').time())].interpolate(limit=10)
        for level in data['Sensor Glucose (mg/dL)']:
            if level > 250:
                hyperglycemia_critical_night += 1
            if level <= 250 and level > 180:
                hyperglycemia_night += 1
            if level <= 180 and level >= 70:
                range_night += 1
            if level <= 150 and level >= 70:
                range_secondary_night += 1
            if level < 70 and level >= 54:
                hypoglycemia_1_night += 1
            if level < 54:
                hypoglycemia_2_night += 1
            count += 1
        manual_hyperglycemia_critical_night += hyperglycemia_critical_night/72
        manual_hyperglycemia_night += hyperglycemia_night/72
        manual_range_night += range_night/72
        manual_range_secondary_night += range_secondary_night/72
        manual_hypoglycemia_1_night += hypoglycemia_1_night/72
        manual_hypoglycemia_2_night += hypoglycemia_2_night/72
        total_manual_night += 1
    else:
        pass
manual_hyperglycemia_critical_day /= total_manual
manual_hyperglycemia_critical_morning /= total_manual_morning
manual_hyperglycemia_critical_night /= total_manual_night
manual_hyperglycemia_day /= total_manual
manual_hyperglycemia_morning /= total_manual_morning 
manual_hyperglycemia_night /= total_manual_night
manual_range_day/= total_manual
manual_range_morning /= total_manual_morning
manual_range_night /= total_manual_night
manual_range_secondary_day/= total_manual
manual_range_secondary_morning /= total_manual_morning 
manual_range_secondary_night /= total_manual_night
manual_hypoglycemia_1_day/= total_manual
manual_hypoglycemia_1_morning /= total_manual_morning
manual_hypoglycemia_1_night /= total_manual_night
manual_hypoglycemia_2_day/= total_manual
manual_hypoglycemia_2_morning /= total_manual_morning
manual_hypoglycemia_2_night /= total_manual_night



In [151]:
a_days = auto_data["Date"].unique()[::-1]

In [178]:
auto_hyperglycemia_critical_day, auto_hyperglycemia_critical_morning, auto_hyperglycemia_critical_night = 0, 0, 0
auto_hyperglycemia_day, auto_hyperglycemia_morning, auto_hyperglycemia_night = 0,0,0
auto_range_day, auto_range_morning, auto_range_night = 0, 0, 0
auto_range_secondary_day, auto_range_secondary_morning, auto_range_secondary_night = 0, 0, 0
auto_hypoglycemia_1_day, auto_hypoglycemia_1_morning, auto_hypoglycemia_1_night = 0, 0, 0
auto_hypoglycemia_2_day, auto_hypoglycemia_2_morning, auto_hypoglycemia_2_night = 0, 0, 0
total_auto = 0
total_auto_morning = 0
total_auto_night = 0

for day in a_days:
    #print(f"Day:{day}, Rows:{len(auto_data[(auto_data['Date'] == day) & (auto_data['Sensor Glucose (mg/dL)'].notna())])}")
    actual_night = len(auto_data[(auto_data['Date'] == day) & (pd.to_datetime(auto_data['Time'],format='%H:%M:%S').dt.time < pd.to_datetime("6:00:00",format='%H:%M:%S').time()) & (auto_data['Sensor Glucose (mg/dL)'].notna())])
    total_night = len(auto_data[(auto_data['Date'] == day) & (pd.to_datetime(auto_data['Time'],format='%H:%M:%S').dt.time < pd.to_datetime("6:00:00",format='%H:%M:%S').time())])
    night_diff = total_night - actual_night
    
    actual_morning = len(auto_data[(auto_data['Date'] == day) & (pd.to_datetime(auto_data['Time'],format='%H:%M:%S').dt.time >= pd.to_datetime("6:00:00",format='%H:%M:%S').time()) & (auto_data['Sensor Glucose (mg/dL)'].notna())])
    total_morning = len(auto_data[(auto_data['Date'] == day) & (pd.to_datetime(auto_data['Time'],format='%H:%M:%S').dt.time >= pd.to_datetime("6:00:00",format='%H:%M:%S').time())])
    morning_diff = total_morning - actual_morning
    
    actual_day = len(auto_data[(auto_data['Date'] == day) & (auto_data['Sensor Glucose (mg/dL)'].notna())])
    total_day = len(auto_data[(auto_data['Date'] == day)])
    day_diff = total_day - actual_day
    
#     print(f"Day:{day}, Rows:{total_day}, Diff:{day_diff}, Morning:{total_morning}, Diff:{morning_diff}, Night:{total_night}, Diff:{night_diff}")
    
    if day_diff <= 10 or total_day == 288:
        count = 1
        
        hyperglycemia_critical_day, hyperglycemia_critical_morning, hyperglycemia_critical_night = 0, 0, 0
        hyperglycemia_day, hyperglycemia_morning, hyperglycemia_night = 0,0,0
        range_day, range_morning, range_night = 0, 0, 0
        range_secondary_day, range_secondary_morning, range_secondary_night = 0, 0, 0
        hypoglycemia_1_day, hypoglycemia_1_morning, hypoglycemia_1_night = 0, 0, 0
        hypoglycemia_2_day, hypoglycemia_2_morning, hypoglycemia_2_night = 0, 0, 0
        
        data = auto_data[(auto_data['Date'] == day)].interpolate(limit=10)
        for level in data['Sensor Glucose (mg/dL)']:
            if level > 250:
                hyperglycemia_critical_day += 1
                if count <= 216: hyperglycemia_critical_morning += 1  
                else: hyperglycemia_critical_night += 1
            if level <= 250 and level > 180:
                hyperglycemia_day += 1
                if count <= 216: hyperglycemia_morning += 1 
                else: hyperglycemia_night += 1
            if level <= 180 and level >= 70:
                range_day += 1
                if count <= 216: range_morning += 1
                else: range_night += 1
            if level <= 150 and level >= 70:
                range_secondary_day += 1
                if count <= 216: range_secondary_morning += 1
                else: range_secondary_night += 1
            if level < 70 and level >= 54:
                hypoglycemia_1_day += 1
                if count <= 216: hypoglycemia_1_morning += 1
                else: hypoglycemia_1_night += 1
            if level < 54:
                hypoglycemia_2_day += 1
                if count <= 216: hypoglycemia_2_morning += 1
                else: hypoglycemia_2_night += 1
            count += 1
        auto_hyperglycemia_critical_day += hyperglycemia_critical_day/288
        auto_hyperglycemia_critical_morning += hyperglycemia_critical_morning/216
        auto_hyperglycemia_critical_night += hyperglycemia_critical_night/72
        auto_hyperglycemia_day += hyperglycemia_day/288
        auto_hyperglycemia_morning += hyperglycemia_morning/216
        auto_hyperglycemia_night += hyperglycemia_night/72
        auto_range_day += range_day/288
        auto_range_morning += range_morning/216
        auto_range_night += range_night/72
        auto_range_secondary_day += range_secondary_day/288
        auto_range_secondary_morning += range_secondary_morning/216
        auto_range_secondary_night += range_secondary_night/72
        auto_hypoglycemia_1_day += hypoglycemia_1_day/288
        auto_hypoglycemia_1_morning += hypoglycemia_1_morning/216
        auto_hypoglycemia_1_night += hypoglycemia_1_night/72
        auto_hypoglycemia_2_day += hypoglycemia_2_day/288
        auto_hypoglycemia_2_morning += hypoglycemia_2_morning/216
        auto_hypoglycemia_2_night += hypoglycemia_2_night/72
        total_auto += 1
        total_auto_morning += 1
        total_auto_night += 1
    elif morning_diff <= 10 or total_morning == 216:
        count = 1
        
        hyperglycemia_critical_day, hyperglycemia_critical_morning, hyperglycemia_critical_night = 0, 0, 0
        hyperglycemia_day, hyperglycemia_morning, hyperglycemia_night = 0,0,0
        range_day, range_morning, range_night = 0, 0, 0
        range_secondary_day, range_secondary_morning, range_secondary_night = 0, 0, 0
        hypoglycemia_1_day, hypoglycemia_1_morning, hypoglycemia_1_night = 0, 0, 0
        hypoglycemia_2_day, hypoglycemia_2_morning, hypoglycemia_2_night = 0, 0, 0
        
        data = auto_data[(auto_data['Date'] == day)& (pd.to_datetime(auto_data['Time'],format='%H:%M:%S').dt.time >= pd.to_datetime("6:00:00",format='%H:%M:%S').time())].interpolate(limit=10)
        for level in data['Sensor Glucose (mg/dL)']:
            if level > 250:
                hyperglycemia_critical_morning += 1  
            if level <= 250 and level > 180:
                hyperglycemia_morning += 1 
            if level <= 180 and level >= 70:
                range_morning += 1
            if level <= 150 and level >= 70:
                range_secondary_morning += 1
            if level < 70 and level >= 54:
                hypoglycemia_1_morning += 1
            if level < 54:
                hypoglycemia_2_morning += 1
            count += 1
        auto_hyperglycemia_critical_morning += hyperglycemia_critical_morning/216
        auto_hyperglycemia_morning += hyperglycemia_morning/216
        auto_range_morning += range_morning/216
        auto_range_secondary_morning += range_secondary_morning/216
        auto_hypoglycemia_1_morning += hypoglycemia_1_morning/216
        auto_hypoglycemia_2_morning += hypoglycemia_2_morning/216
        total_auto_morning += 1
    elif night_diff <= 10 or total_night == 72:
        count = 1
        
        hyperglycemia_critical_day, hyperglycemia_critical_morning, hyperglycemia_critical_night = 0, 0, 0
        hyperglycemia_day, hyperglycemia_morning, hyperglycemia_night = 0,0,0
        range_day, range_morning, range_night = 0, 0, 0
        range_secondary_day, range_secondary_morning, range_secondary_night = 0, 0, 0
        hypoglycemia_1_day, hypoglycemia_1_morning, hypoglycemia_1_night = 0, 0, 0
        hypoglycemia_2_day, hypoglycemia_2_morning, hypoglycemia_2_night = 0, 0, 0
        
        data = auto_data[(auto_data['Date'] == day) & (pd.to_datetime(auto_data['Time'],format='%H:%M:%S').dt.time < pd.to_datetime("6:00:00",format='%H:%M:%S').time())].interpolate(limit=10)
        for level in data['Sensor Glucose (mg/dL)']:
            if level > 250:
                hyperglycemia_critical_night += 1
            if level <= 250 and level > 180:
                hyperglycemia_night += 1
            if level <= 180 and level >= 70:
                range_night += 1
            if level <= 150 and level >= 70:
                range_secondary_night += 1
            if level < 70 and level >= 54:
                hypoglycemia_1_night += 1
            if level < 54:
                hypoglycemia_2_night += 1
            count += 1
        auto_hyperglycemia_critical_night += hyperglycemia_critical_night/72
        auto_hyperglycemia_night += hyperglycemia_night/72
        auto_range_night += range_night/72
        auto_range_secondary_night += range_secondary_night/72
        auto_hypoglycemia_1_night += hypoglycemia_1_night/72
        auto_hypoglycemia_2_night += hypoglycemia_2_night/72
        total_auto_night += 1
    else:
        pass
auto_hyperglycemia_critical_day /= total_auto
auto_hyperglycemia_critical_morning /= total_auto_morning
auto_hyperglycemia_critical_night /= total_auto_night
auto_hyperglycemia_day /= total_auto
auto_hyperglycemia_morning /= total_auto_morning 
auto_hyperglycemia_night /= total_auto_night
auto_range_day/= total_auto
auto_range_morning /= total_auto_morning
auto_range_night /= total_auto_night
auto_range_secondary_day/= total_auto
auto_range_secondary_morning /= total_auto_morning 
auto_range_secondary_night /= total_auto_night
auto_hypoglycemia_1_day/= total_auto
auto_hypoglycemia_1_morning /= total_auto_morning
auto_hypoglycemia_1_night /= total_auto_night
auto_hypoglycemia_2_day/= total_auto
auto_hypoglycemia_2_morning /= total_auto_morning
auto_hypoglycemia_2_night /= total_auto_night

#     if total == 288 and diff == 10:
#         print(288 - len(auto_data[(auto_data['Date'] == day) & (auto_data['Sensor Glucose (mg/dL)'].notna())]))
#         data = auto_data[(auto_data['Date'] == day)]
#         new_auto_data = data[data['Date'] == day].interpolate(limit=10)
#         print(288 - len(new_auto_data[(new_auto_data['Date'] == day) & (new_auto_data['Sensor Glucose (mg/dL)'].notna())]))

In [153]:
import csv

In [179]:
f = open('./Results.csv', 'w')
writer = csv.writer(f)

writer.writerow([manual_hyperglycemia_night * 100, manual_hyperglycemia_critical_night * 100, manual_range_night * 100, manual_range_secondary_night * 100, manual_hypoglycemia_1_night * 100, manual_hypoglycemia_2_night * 100, manual_hyperglycemia_morning * 100, manual_hyperglycemia_critical_morning * 100, manual_range_morning * 100, manual_range_secondary_morning * 100, manual_hypoglycemia_1_morning * 100, manual_hypoglycemia_2_morning * 100, manual_hyperglycemia_day * 100, manual_hyperglycemia_critical_day * 100, manual_range_day * 100, manual_range_secondary_day * 100, manual_hypoglycemia_1_day * 100, manual_hypoglycemia_2_day * 100])
writer.writerow([auto_hyperglycemia_night * 100, auto_hyperglycemia_critical_night * 100, auto_range_night * 100, auto_range_secondary_night * 100, auto_hypoglycemia_1_night * 100, auto_hypoglycemia_2_night * 100, auto_hyperglycemia_morning * 100, auto_hyperglycemia_critical_morning * 100, auto_range_morning * 100, auto_range_secondary_morning * 100, auto_hypoglycemia_1_morning * 100, auto_hypoglycemia_2_morning * 100, auto_hyperglycemia_day * 100, auto_hyperglycemia_critical_day * 100, auto_range_day * 100, auto_range_secondary_day * 100, auto_hypoglycemia_1_day * 100, auto_hypoglycemia_2_day * 100])

f.close()

In [176]:
# for day in a_days:
#     #print(f"Day:{day}, Rows:{len(auto_data[(auto_data['Date'] == day) & (auto_data['Sensor Glucose (mg/dL)'].notna())])}")
#     actual_night = len(auto_data[(auto_data['Date'] == day) & (pd.to_datetime(auto_data['Time'],format='%H:%M:%S').dt.time < pd.to_datetime("6:00:00",format='%H:%M:%S').time()) & (auto_data['Sensor Glucose (mg/dL)'].notna())])
#     total_night = len(auto_data[(auto_data['Date'] == day) & (pd.to_datetime(auto_data['Time'],format='%H:%M:%S').dt.time < pd.to_datetime("6:00:00",format='%H:%M:%S').time())])
#     night_diff = total_night - actual_night
    
#     actual_morning = len(auto_data[(auto_data['Date'] == day) & (pd.to_datetime(auto_data['Time'],format='%H:%M:%S').dt.time >= pd.to_datetime("6:00:00",format='%H:%M:%S').time()) & (auto_data['Sensor Glucose (mg/dL)'].notna())])
#     total_morning = len(auto_data[(auto_data['Date'] == day) & (pd.to_datetime(auto_data['Time'],format='%H:%M:%S').dt.time >= pd.to_datetime("6:00:00",format='%H:%M:%S').time())])
#     morning_diff = total_morning - actual_morning
    
#     actual_day = len(auto_data[(auto_data['Date'] == day) & (auto_data['Sensor Glucose (mg/dL)'].notna())])
#     total_day = len(auto_data[(auto_data['Date'] == day)])
#     day_diff = total_day - actual_day
    
# #     print(f"Day:{day}, Rows:{total_day}, Diff:{day_diff}, Morning:{total_morning}, Diff:{morning_diff}, Night:{total_night}, Diff:{night_diff}")
    
#     if day_diff <= 10 or total_day == 288:
#         count = 1
        
#         hyperglycemia_critical_day, hyperglycemia_critical_morning, hyperglycemia_critical_night = 0, 0, 0
#         hyperglycemia_day, hyperglycemia_morning, hyperglycemia_night = 0,0,0
#         range_day, range_morning, range_night = 0, 0, 0
#         range_secondary_day, range_secondary_morning, range_secondary_night = 0, 0, 0
#         hypoglycemia_1_day, hypoglycemia_1_morning, hypoglycemia_1_night = 0, 0, 0
#         hypoglycemia_2_day, hypoglycemia_2_morning, hypoglycemia_2_night = 0, 0, 0
        
#         data = auto_data[(auto_data['Date'] == day)].interpolate(limit=10)
#         for level in data['Sensor Glucose (mg/dL)']:
#             if level > 250:
#                 hyperglycemia_critical_day += 1
#                 if count <= 216: hyperglycemia_critical_morning += 1  
#                 else: hyperglycemia_critical_night += 1
#             if level <= 250 and level > 180:
#                 hyperglycemia_day += 1
#                 if count <= 216: hyperglycemia_morning += 1 
#                 else: hyperglycemia_night += 1
#             if level <= 180 and level >= 70:
#                 range_day += 1
#                 if count <= 216: range_morning += 1
#                 else: range_night += 1
#             if level <= 150 and level >= 70:
#                 range_secondary_day += 1
#                 if count <= 216: range_secondary_morning += 1
#                 else: range_secondary_night += 1
#             if level < 70 and level >= 54:
#                 hypoglycemia_1_day += 1
#                 if count <= 216: hypoglycemia_1_morning += 1
#                 else: hypoglycemia_1_night += 1
#             if level < 54:
#                 hypoglycemia_2_day += 1
#                 if count <= 216: hypoglycemia_2_morning += 1
#                 else: hypoglycemia_2_night += 1
#             count += 1
#         auto_hyperglycemia_critical_day += hyperglycemia_critical_day/288
#         auto_hyperglycemia_critical_morning += hyperglycemia_critical_morning/216
#         auto_hyperglycemia_critical_night += hyperglycemia_critical_night/72
#         auto_hyperglycemia_day += hyperglycemia_day/288
#         auto_hyperglycemia_morning += hyperglycemia_morning/216
#         auto_hyperglycemia_night += hyperglycemia_night/72
#         auto_range_day += range_day/288
#         auto_range_morning += range_morning/216
#         auto_range_night += range_night/72
#         auto_range_secondary_day += range_secondary_day/288
#         auto_range_secondary_morning += range_secondary_morning/216
#         auto_range_secondary_night += range_secondary_night/72
#         auto_hypoglycemia_1_day += hypoglycemia_1_day/288
#         auto_hypoglycemia_1_morning += hypoglycemia_1_morning/216
#         auto_hypoglycemia_1_night += hypoglycemia_1_night/72
#         auto_hypoglycemia_2_day += hypoglycemia_2_day/288
#         auto_hypoglycemia_2_morning += hypoglycemia_2_morning/216
#         auto_hypoglycemia_2_night += hypoglycemia_2_night/72
#         total_auto += 1
#         total_auto_morning += 1
#         total_auto_night += 1
#     elif morning_diff <= 10 or total_morning == 216:
#         count = 1
        
#         hyperglycemia_critical_day, hyperglycemia_critical_morning, hyperglycemia_critical_night = 0, 0, 0
#         hyperglycemia_day, hyperglycemia_morning, hyperglycemia_night = 0,0,0
#         range_day, range_morning, range_night = 0, 0, 0
#         range_secondary_day, range_secondary_morning, range_secondary_night = 0, 0, 0
#         hypoglycemia_1_day, hypoglycemia_1_morning, hypoglycemia_1_night = 0, 0, 0
#         hypoglycemia_2_day, hypoglycemia_2_morning, hypoglycemia_2_night = 0, 0, 0
        
#         data = auto_data[(auto_data['Date'] == day)& (pd.to_datetime(auto_data['Time'],format='%H:%M:%S').dt.time >= pd.to_datetime("6:00:00",format='%H:%M:%S').time())].interpolate(limit=10)
#         for level in data['Sensor Glucose (mg/dL)']:
#             if level > 250:
#                 hyperglycemia_critical_morning += 1  
#             if level <= 250 and level > 180:
#                 hyperglycemia_morning += 1 
#             if level <= 180 and level >= 70:
#                 range_morning += 1
#             if level <= 150 and level >= 70:
#                 range_secondary_morning += 1
#             if level < 70 and level >= 54:
#                 hypoglycemia_1_morning += 1
#             if level < 54:
#                 hypoglycemia_2_morning += 1
#             count += 1
#         auto_hyperglycemia_critical_morning += hyperglycemia_critical_morning/216
#         auto_hyperglycemia_morning += hyperglycemia_morning/216
#         auto_range_morning += range_morning/216
#         auto_range_secondary_morning += range_secondary_morning/216
#         auto_hypoglycemia_1_morning += hypoglycemia_1_morning/216
#         auto_hypoglycemia_2_morning += hypoglycemia_2_morning/216
#         total_auto_morning += 1
#     elif night_diff <= 10 or total_night == 72:
#         count = 1
        
#         hyperglycemia_critical_day, hyperglycemia_critical_morning, hyperglycemia_critical_night = 0, 0, 0
#         hyperglycemia_day, hyperglycemia_morning, hyperglycemia_night = 0,0,0
#         range_day, range_morning, range_night = 0, 0, 0
#         range_secondary_day, range_secondary_morning, range_secondary_night = 0, 0, 0
#         hypoglycemia_1_day, hypoglycemia_1_morning, hypoglycemia_1_night = 0, 0, 0
#         hypoglycemia_2_day, hypoglycemia_2_morning, hypoglycemia_2_night = 0, 0, 0
        
#         data = auto_data[(auto_data['Date'] == day) & (pd.to_datetime(auto_data['Time'],format='%H:%M:%S').dt.time < pd.to_datetime("6:00:00",format='%H:%M:%S').time())].interpolate(limit=10)
#         for level in data['Sensor Glucose (mg/dL)']:
#             if level > 250:
#                 hyperglycemia_critical_night += 1
#             if level <= 250 and level > 180:
#                 hyperglycemia_night += 1
#             if level <= 180 and level >= 70:
#                 range_night += 1
#             if level <= 150 and level >= 70:
#                 range_secondary_night += 1
#             if level < 70 and level >= 54:
#                 hypoglycemia_1_night += 1
#             if level < 54:
#                 hypoglycemia_2_night += 1
#             count += 1
#         auto_hyperglycemia_critical_night += hyperglycemia_critical_night/72
#         auto_hyperglycemia_night += hyperglycemia_night/72
#         auto_range_night += range_night/72
#         auto_range_secondary_night += range_secondary_night/72
#         auto_hypoglycemia_1_night += hypoglycemia_1_night/72
#         auto_hypoglycemia_2_night += hypoglycemia_2_night/72
#         total_auto_night += 1
#     else:
#         pass
# auto_hyperglycemia_critical_day /= total_auto
# auto_hyperglycemia_critical_morning /= total_auto_morning
# auto_hyperglycemia_critical_night /= total_auto_night
# auto_hyperglycemia_day /= total_auto
# auto_hyperglycemia_morning /= total_auto_morning 
# auto_hyperglycemia_night /= total_auto_night
# auto_range_day/= total_auto
# auto_range_morning /= total_auto_morning
# auto_range_night /= total_auto_night
# auto_range_secondary_day/= total_auto
# auto_range_secondary_morning /= total_auto_morning 
# auto_range_secondary_night /= total_auto_night
# auto_hypoglycemia_1_day/= total_auto
# auto_hypoglycemia_1_morning /= total_auto_morning
# auto_hypoglycemia_1_night /= total_auto_night
# auto_hypoglycemia_2_day/= total_auto
# auto_hypoglycemia_2_morning /= total_auto_morning
# auto_hypoglycemia_2_night /= total_auto_night


Day:8/9/2017, Rows:190, Diff:0, Morning:190, Diff:0, Night:0, Diff:0
Day:8/10/2017, Rows:280, Diff:83, Morning:208, Diff:52, Night:72, Diff:31
Day:8/11/2017, Rows:288, Diff:1, Morning:216, Diff:0, Night:72, Diff:1
Day:8/12/2017, Rows:288, Diff:0, Morning:216, Diff:0, Night:72, Diff:0
Day:8/13/2017, Rows:288, Diff:20, Morning:216, Diff:20, Night:72, Diff:0
Day:8/14/2017, Rows:289, Diff:20, Morning:216, Diff:0, Night:73, Diff:20
Day:8/15/2017, Rows:288, Diff:3, Morning:216, Diff:0, Night:72, Diff:3
Day:8/16/2017, Rows:288, Diff:24, Morning:216, Diff:24, Night:72, Diff:0
Day:8/17/2017, Rows:281, Diff:44, Morning:209, Diff:44, Night:72, Diff:0
Day:8/18/2017, Rows:288, Diff:3, Morning:216, Diff:0, Night:72, Diff:3
Day:8/19/2017, Rows:288, Diff:0, Morning:216, Diff:0, Night:72, Diff:0
Day:8/20/2017, Rows:288, Diff:1, Morning:216, Diff:1, Night:72, Diff:0
Day:8/21/2017, Rows:288, Diff:48, Morning:216, Diff:48, Night:72, Diff:0
Day:8/22/2017, Rows:282, Diff:112, Morning:210, Diff:73, Night:72,

Day:12/1/2017, Rows:288, Diff:0, Morning:216, Diff:0, Night:72, Diff:0
Day:12/2/2017, Rows:288, Diff:2, Morning:216, Diff:2, Night:72, Diff:0
Day:12/3/2017, Rows:288, Diff:0, Morning:216, Diff:0, Night:72, Diff:0
Day:12/4/2017, Rows:288, Diff:3, Morning:216, Diff:0, Night:72, Diff:3
Day:12/5/2017, Rows:288, Diff:53, Morning:216, Diff:39, Night:72, Diff:14
Day:12/6/2017, Rows:288, Diff:7, Morning:216, Diff:4, Night:72, Diff:3
Day:12/7/2017, Rows:281, Diff:34, Morning:209, Diff:34, Night:72, Diff:0
Day:12/8/2017, Rows:288, Diff:39, Morning:216, Diff:36, Night:72, Diff:3
Day:12/9/2017, Rows:285, Diff:31, Morning:213, Diff:31, Night:72, Diff:0
Day:12/10/2017, Rows:288, Diff:3, Morning:216, Diff:0, Night:72, Diff:3
Day:12/11/2017, Rows:288, Diff:0, Morning:216, Diff:0, Night:72, Diff:0
Day:12/12/2017, Rows:288, Diff:0, Morning:216, Diff:0, Night:72, Diff:0
Day:12/13/2017, Rows:287, Diff:62, Morning:215, Diff:62, Night:72, Diff:0
Day:12/14/2017, Rows:288, Diff:39, Morning:216, Diff:7, Night: